databricks notebook2: clean raw data

In [ ]:
from pyspark.sql.functions import col

In [ ]:
# define Gen2 paths
trip_csv_path = "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/bronze/trip_transactions/Trip_Transaction_Table.csv"
rating_csv_path = "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/bronze/ride_ratings/Ride_Rating_Table.csv"

In [ ]:
# read raw data
trip_df = spark.read.csv(
    trip_csv_path,
    header=True,
    inferSchema=True
)

In [ ]:
rating_df =spark.read.csv(
    rating_csv_path,
    header=True,
    inferSchema=True
)

In [ ]:
# clean trip data
trip_df_cleaned = trip_df.filter(col("trip_id").isNotNull()) \
    .withColumn("trip_start_timestamp", to_timestamp("trip_start_timestamp")) \
    .withColumn("trip_end_timestamp", to_timestamp("trip_end_timestamp")) \
    .withColumn("total_distance", col("total_distance").cast("double")) \
    .withColumn("total_fare", col("total_fare").cast("double")) \
    .withColumn("delay_start_time_mins", col("delay_start_time_mins").cast("int"))

In [ ]:
# clean rating data
rating_df_cleaned = rating_df \
    .filter(col('trip_id').isNotNull()) \
    .withColumn('customer_rating',col('customer_rating').cast('int')) \
    .withColumn('driver_rating',col('driver_rating').cast('int'))

In [ ]:
# define delta path for cleaned files
path1 = "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/trip_transactions"
path2 = "abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/ride_ratings"

In [ ]:
# write data to delta tables
trip_df_cleaned.write.format('delta').mode('overwrite').save(path1)
rating_df_cleaned.write.format('delta').mode('overwrite').save(path2)